In [27]:
import pandas as pd
from sklearn import datasets  # imports datasets from scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor  # decision tree regression
from sklearn import metrics
import numpy as np

In [38]:
from warnings import simplefilter

simplefilter(action='ignore', category=FutureWarning)

In [44]:
import sys

try:
    data = datasets.load_boston()  # loads Boston dataset from datasets library
    # parsed = json.loads(data)
    # parsed = json.loads(data.data)
    # parsed = json.load(data.data)
    parsed = json.loads(data.DESCR)
    print(json.dumps(parsed, indent=4, sort_keys=True))


    def pp_json(json_thing, sort=True, indents=4):
        if type(json_thing) is str:
            print(json.dumps(json.loads(json_thing), sort_keys=sort, indent=indents))
        else:
            print(json.dumps(json_thing, sort_keys=sort, indent=indents))
        return None


    pp_json(data)

    data
except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print("\nType", exc_type)
    print("\nErr:", exc_obj)
    print("\nLine:", exc_tb.tb_lineno)



Type <class 'json.decoder.JSONDecodeError'>

Err: Expecting value: line 1 column 1 (char 0)

Line: 8


In [7]:
# define the data/predictors as the pre-set feature names
X = pd.DataFrame(data.data, columns=data.feature_names)
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


### MEDV

Median value of owner-occupied homes in $1000's

In [8]:
# Put the target (housing value -- MEDV) in another DataFrame
Y = pd.DataFrame(data.target, columns=["MEDV"])
# data.target
Y.head()

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [46]:
# SPLIT THE DATA

# 25% hold out for testing
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.25, random_state=25)

# 379 rows x 13 columns
# print('\nX_train\n')
# print(X_train)

# 127 rows x 13 columns
# print('\nX_test\n')
# print(X_test)

# 379 rows x 1 columns
# print('\ny_train\n')
# print(y_train)

# 127 rows x 1 columns
# print('\ny_test\n')
# print(y_test)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((379, 13), (127, 13), (379, 1), (127, 1))

## Instantiate decision tree model (depth = 2)

If the maximum depth of the tree (controlled by the max_depth parameter) is set too high, overfitting may happen.

In [14]:
regressor = DecisionTreeRegressor(max_depth=2)

# FIT = Build a decision tree regressor from the training set (X, y).
regressor.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=2)

In [17]:
# predict Y values- 25% training data
Y_pred = regressor.predict(X_test)

Y_pred[0]

14.800000000000002

In [18]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, Y_pred)  # MSE

# ONLY GOOD FOR PYTHON 2 - ce n'est pas vrai

32.299068640612354

## Use cross-validation

When we can't split our data into training and testing sets

In [19]:
regression_tree = DecisionTreeRegressor(min_samples_split=30, min_samples_leaf=10, random_state=0)
regression_tree


DecisionTreeRegressor(min_samples_leaf=10, min_samples_split=30, random_state=0)

## Cross-validation: evaluating estimator performance

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# crossvalidation = KFold(n=X.shape[0], n_folds=10, shuffle=True, random_state=1) # 10-fold CV

# class sklearn.model_selection.KFold(n_splits=’warn’, shuffle=False, random_state=None)
crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)  # 10 fold CV
crossvalidation


KFold(n_splits=10, random_state=1, shuffle=True)

In [22]:
regression_tree.fit(X, Y)


DecisionTreeRegressor(min_samples_leaf=10, min_samples_split=30, random_state=0)

In [33]:
# sorted(sklearn.metrics.SCORERS.keys())
sorted(metrics.SCORERS.keys())

# mean_squared_error
score = np.mean(cross_val_score(regression_tree, X, Y, scoring='neg_mean_squared_error', cv=crossvalidation, n_jobs=1))

# 10-fold CV based MSE

In [34]:
print("Mean squared error: %.3f" % abs(score))

Mean squared error: 20.107


## Decide on the depth

In [36]:
for depth in range(1, 10):
    regression_tree2 = DecisionTreeRegressor(min_samples_split=30, min_samples_leaf=10, random_state=0)

    if regression_tree2.fit(X, Y).tree_.max_depth < depth:
        break
        score = np.mean(cross_val_score(tree_classifier, X, Y, scoring="mean_squared_error", cv=crossvalidation, n_jobs=1))

    print("Depth: %i Accuracy: %.3f" % (depth, score))


Depth: 1 Accuracy: -20.107
Depth: 2 Accuracy: -20.107
Depth: 3 Accuracy: -20.107
Depth: 4 Accuracy: -20.107
Depth: 5 Accuracy: -20.107
Depth: 6 Accuracy: -20.107
Depth: 7 Accuracy: -20.107
Depth: 8 Accuracy: -20.107
Depth: 9 Accuracy: -20.107
